In [1]:
# import Python
import os
from dotenv import load_dotenv
from numpy import vectorize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sty import fg
import json
import sys

# import fichier
from Functions import json_decoder as jsd

# Mise en place de l'environnement de travail 
# - Récupération de l'extension souhaitée 
# - Tri des fichiers à analyser
s=""
sample_files = []
load_dotenv()
Space_Cleared_Exts = os.environ["EXTENSIONS"].replace(" ", "")

for ext in os.environ["EXTENSIONS"] :
    if(ext==",") :
        ext="["
        for doc in os.listdir("./FilesToTest/") :
            if doc.endswith(str(s)) :
                sample_files.append(''.join(doc))
        s = ""
    if(ext!="[" and ext!="]") :
        s+=ext
for doc in os.listdir("./FilesToTest/") :
    if doc.endswith(str(s.replace(" ",""))) :
        sample_files.append(''.join(doc))

# sample_contents contient tous les codes de tous les fichiers .py
sample_contents = [open("./FilesToTest/"+File).read() for File in sample_files]


if (len(sample_contents) <= 1):
    sys.exit("Vous devez mettre un dossier avec un minimum 2 fichiers Python")


In [2]:
Commentary_Content = []
Var_Content = []
Equal_Content = []
Loop_Content = []

Sample_Content_Double = []
Sample_Content_Without_CommsLoops = []
Sample_Content_Without_CommsLoopsVar = []
# ---------------------------------------------------------------------------------
# Parcourt notre texte et enleve les commentaires si le texte vient d'un fichier JS
# ---------------------------------------------------------------------------------

# PurgeCommentary
#  
def PurgeCommentary(com_format,ext_com_format, syntax) :
    Sample_ContentsBis = sample_contents[syntax].splitlines(True)
    Sample_Content_SubFile = []
    Commentary_SubFile = []
    Long_Comment = False
    Append_Lines = False    
    Stop_Ext_Com = "None"
    for lines in Sample_ContentsBis :
        #print(lines)
        for syntax in com_format :
            for ext in ext_com_format :
                if(Stop_Ext_Com in lines):
                    Long_Comment = False
                    Append_Lines = True
                if(Long_Comment) :
                    Append_Lines = True
                if(syntax in lines) :
                    Append_Lines = True
                if(ext in lines):
                    Append_Lines = True
                    Long_Comment = True
                    if(ext=="/*") :
                        Stop_Ext_Com = "*/"
        if(Append_Lines == False and Stop_Ext_Com not in lines and Long_Comment == False):
            Sample_Content_SubFile.append(lines)
        if(Append_Lines and Long_Comment == False) :
            Commentary_SubFile.append(lines)
            Append_Lines = False
        elif(Append_Lines and Long_Comment) :
            Commentary_SubFile.append(lines)
    Sample_Content_Double.append(Sample_Content_SubFile)
    Commentary_Content.append(Commentary_SubFile)

In [3]:
def PurgeLoops(format, index) :
    Sample_Content_SubFile = []
    Loop_SubFile = []
    IsLoop = False
    Append_Lines = False
    Stop_Loop_Text = "None"
    for lines in Sample_Content_Double[index] :
        for syntax in format :
            if(Stop_Loop_Text in lines):
                Append_Lines = True
                IsLoop = False
            if(IsLoop) :
                Append_Lines = True
            if(syntax in lines) :
                Append_Lines = True
                IsLoop = True
                if(syntax=="{") :
                    Stop_Loop_Text = "}"
        if(Append_Lines == False and Stop_Loop_Text not in lines and IsLoop == False):
            Sample_Content_SubFile.append(lines) 
        if(Append_Lines and IsLoop == False) : 
            Loop_SubFile.append(lines)
            Append_Lines = False
        elif(Append_Lines and IsLoop) :
            Loop_SubFile.append(lines)
    Sample_Content_Without_CommsLoops.append(Sample_Content_SubFile)
    Loop_Content.append(Loop_SubFile)

In [4]:
def PurgeVariables(format, index) :
    Sample_Content_SubFile = []
    Var_SubFile = []
    Append_Lines = False
    for lines in Sample_Content_Without_CommsLoops[index] :
        for syntax in format : 
            if(syntax in lines) :
                Append_Lines = True
        if(Append_Lines) :
            Var_SubFile.append(lines)
            Append_Lines = False
        else :
                Sample_Content_SubFile.append(lines)
    Sample_Content_Without_CommsLoopsVar.append(Sample_Content_SubFile)
    Var_Content.append(Var_SubFile)

In [5]:
Code_Struct = {}
def CheckStruct(format_dict, syntax) :
    Var_Index = []
    Loop_Index = []
    Com_Index = []
    append_Lines_Var = False
    append_Lines_Loop = False
    append_Lines_Com = False
    long_Com = False
    stop_LC = "None"
    #print(sample_files[syntax])
    Sample_ContentsBis = sample_contents[syntax].splitlines(True)
    for index in range(len(Sample_ContentsBis)) :
        for exc in format_dict['var'] :
            if(exc in Sample_ContentsBis[index]) :
                append_Lines_Var = True
        for exc in format_dict['loop'] :
            if(exc in Sample_ContentsBis[index]) :
                append_Lines_Loop = True
                
        for exc in format_dict['com'] :
            if(exc in Sample_ContentsBis[index]) :
                append_Lines_Com = True
                
        if(Sample_ContentsBis[index] == stop_LC) :
            long_Com = False
            append_Lines_Com = True
            
        if(long_Com == True):
            append_Lines_Com = True
            
        for exc in format_dict['extended_com'] :
            if(exc in Sample_ContentsBis[index]) :
                append_Lines_Com = True
                long_Com = True
                if(exc=="/*"):
                    stop_LC = "*/"
                    
        if(append_Lines_Var) :
            Var_Index.append(str(index))
            append_Lines_Var = False 
        if(append_Lines_Loop) :
            Loop_Index.append(str(index))
            append_Lines_Loop = False
        if(append_Lines_Com and long_Com == False) : 
            Com_Index.append(str(index))
            append_Lines_Com = False
        elif(append_Lines_Com and long_Com ==True) : 
            Com_Index.append(str(index))
            append_Lines_Com = False
            
    Code_Struct[sample_files[syntax]] = {"VAR": Var_Index, "LOOP" : Loop_Index, "COM": Com_Index}
    #print(Code_Struct)

In [6]:
    for index in range(len(sample_files)) :
        increment_text = False
        temp_text = ""
        for j in range(len(sample_files[index])) :
            if(increment_text) :
                temp_text += (sample_files[index][j]).upper()
            if(sample_files[index][j] == ".") :
                increment_text = True
        format_dict = jsd.decodeFormat(temp_text)
        PurgeCommentary(format_dict['com'],format_dict['extended_com'],index)
        PurgeLoops(format_dict['loop'],index)
        PurgeVariables(format_dict['var'],index)
        CheckStruct(format_dict, index)
    
        #print(Sample_Content_Without_CommsLoopsVar)      

In [7]:
All_Content = []
All_Content_Name = []


#print("Commentary Content : \n")
#print(Commentary_Content)
#print("\nLoop Content : \n")
#print(Loop_Content)
#print("\nVar Content : \n")
#print(Var_Content)

All_Content.append(Commentary_Content)
All_Content.append(Var_Content)
All_Content.append(Loop_Content)

All_Content_Name.append("Commentary")
All_Content_Name.append("Loop")
All_Content_Name.append("Var")

print(All_Content)



[[['#d sdqsd\n'], [], []], [[], [], ['fruits = ["apple", "banana", "cherry"]\n']], [[], [], ['for x in fruits:\n', '  print(x)']]]


In [8]:
# vectorize = lambda Text: TfidfVectorizer().fit_transform(Text).toarray()

def vectorize(Text) :
    return TfidfVectorizer(analyzer="char").fit_transform(Text).toarray()

# similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

def similarity(doc1,doc2) :
    return cosine_similarity([doc1,doc2])

In [9]:
def check_plagiarism(TestFiles):
    vectors = vectorize(TestFiles)
    s_vectors = list(zip(sample_files, vectors))
    results = set()
    for sample_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((sample_a, text_vector_a))
        del new_vectors[current_index]
        for sample_b, text_vector_b in new_vectors:
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            sample_pair = sorted((sample_a, sample_b))
            score = sample_pair[0], sample_pair[1], sim_score
            results.add(score)
    return results

In [10]:
def ExecutePlagiarismChecker(ArrayToTest):
    for array_index in range(len(ArrayToTest)):
        temp_text = ""
        for item_index in range(len(ArrayToTest[array_index])):
            temp_text += ArrayToTest[array_index][item_index]
        ArrayToTest[array_index] = temp_text
    return check_plagiarism(ArrayToTest)
scoring_Dict = {}

PreJson = {}

for i in range(len(sample_files)):
    PreJson[sample_files[i]] = {}
    for j in range(len(sample_files)):
        if (i != j):
            PreJson[sample_files[i]][sample_files[j]] = {}

for index in range(len(All_Content)) :  
    for score in ExecutePlagiarismChecker(All_Content[index]) :
        result = (','.join(map(str,(score)))).split(',')
        if index == 0 :
            PreJson[result[0]][result[1]]["Commentary"] = result[2]
            PreJson[result[1]][result[0]]["Commentary"] = result[2]
        elif index == 1 : 
            PreJson[result[0]][result[1]]["Loop"] = result[2]
            PreJson[result[1]][result[0]]["Loop"] = result[2]
        else : 
            PreJson[result[0]][result[1]]["Var"] = result[2]
            PreJson[result[1]][result[0]]["Var"] = result[2]

for index in range(len(sample_contents)):
   for index2 in range(len(sample_contents)):
        if (index  != index2):
            testPlagiat = ExecutePlagiarismChecker([sample_contents[index], sample_contents[index2]])
            mapTestPlagiat = (','.join(map(str,(testPlagiat)))).split(',')
            score = mapTestPlagiat[2]
            score = score.replace(')', "")
            PreJson[sample_files[index]][sample_files[index2]]["Score"] = score
            PreJson[sample_files[index2]][sample_files[index]]["Score"] = score

jsonString = json.dumps(PreJson)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()